# import libraries
코랩 환경에서 실행하였습니다. 데이터 다운로드 및 처리에 필요한 라이브러리를 다운로드 받고 임포트합니다.

In [ ]:
!pip install ijson
!pip install fastparquet

In [ ]:
from collections import Counter, defaultdict
import heapq

from huggingface_hub import hf_hub_download
from tqdm import tqdm
import dask.dataframe as dd
import pandas as pd

# 그래프 데이터 생성

코랩 환경에서 약 35분 소요되었습니다. 이 부분은 뒤의 데이터 로딩 코드로 대체할 수 있습니다.

## 데이터 다운로드
나무위키 문서 데이터셋을 다운로드하고 dask 라이브러리를 사용해 불러옵니다.

데이터셋 크기가 매우 커서 pandas를 사용하여 한 번에 불러올 수 없습니다.

In [ ]:
# Download with progress bar
file_path = hf_hub_download(
    repo_id="heegyu/namuwiki", repo_type='dataset', filename="namuwiki_20210301.parquet")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


namuwiki_20210301.parquet:   0%|          | 0.00/3.03G [00:00<?, ?B/s]

In [ ]:
# Load the Parquet file
ddf = dd.read_parquet(file_path)

## 데이터 처리



나무위키 문법에서 링크를 나타내는 `[[...]]` 부분을 추출하는 함수입니다.

각주(`[*...]`) 안은 무시합니다.

In [ ]:
def extract_double_bracket_links(text):
    """
    Extracts [[...]]-style links from text with filtering rules:
    - Only the part before '|' is kept, if present.
    - Links inside [* ... ] blocks are ignored.
    - Links starting with '파일:', 'http://', or 'https://' are ignored.

    Parameters:
        text (str): Input string.

    Returns:
        dict: Dictionary of valid link targets with occurrence counts.
    """
    results = []
    i = 0
    n = len(text)
    in_footnote = False

    while i < n:
        if not in_footnote and text[i:i+2] == "[*":
            in_footnote = True
            i += 2

        elif text[i:i+2] == "[[":
            end = text.find("]]", i + 2)
            if end == -1:
                break  # No matching ]]
            content = text[i+2:end]
            i = end + 2

            if in_footnote:
                continue

            # Remove after | if exists
            if "|" in content:
                content = content.split("|", 1)[0]

            # Filter unwanted prefixes
            if not (
                content.startswith("파일:") or
                content.startswith("http://") or
                content.startswith("https://")
            ):
                results.append(content)


        elif in_footnote and text[i] == "]":
            in_footnote = False
            i += 1

        else:
            i += 1

    return dict(Counter(results))


빠른 접근을 위해 id와 title 각각을 인덱스로 하는 Series를 만듭니다.

In [ ]:
s_id_title = ddf['title'].compute().reset_index(drop=True)
s_title_id = s_id_title.reset_index().set_index('title', verify_integrity=True)['index'].sort_index()

문서 텍스트 데이터를 분석하여 문서 간 연결 관계를 저장하는 DataFrame을 만듭니다.

In [ ]:
def create_link_edges_dataframe_dask(df_dask):
    # Use a closure to pass the mapping and link extractor
    def partition_func(partition_df):
        print('running partition')
        # records = []

        # for row in partition_df.itertuples(index=False):
        #     from_title = row.title
        #     text = row.text
        #     links = extract_double_bracket_links(text)

        #     for to_title, count in links.items():
        #         if to_title in s_title_id.index:
        #             records.append((s_title_id.loc[from_title], s_title_id.loc[to_title], count))

        records = (
            (s_title_id.loc[row.title], s_title_id.loc[to_title], count)
            for row in partition_df.itertuples(index=False)
            for to_title, count in extract_double_bracket_links(row.text).items()
            if to_title in s_title_id.index
        )

        return pd.DataFrame(records, columns=['from_title', 'to_title', 'count'])

    # Apply per partition
    link_df = df_dask.map_partitions(partition_func, meta={'from_title': int, 'to_title': int, 'count': int})

    return link_df


In [ ]:
edges_df = create_link_edges_dataframe_dask(ddf).compute()

running partition
running partition
running partition
running partition
running partition
running partition
running partition
running partition
running partition
running partition
running partition
running partition
running partition
running partition
running partition
running partition
running partition
running partition
running partition
running partition
running partition
running partition
running partition
running partition
running partition
running partition
running partition
running partition
running partition
running partition
running partition
running partition
running partition
running partition


In [ ]:
edges_df

,from_title,to_title,count
0,1,143751,2
1,1,128620,1
2,1,128612,2
3,1,118944,1
4,1,77175,1
...,...,...,...
20741,867022,203442,1
20742,867022,283833,1
20743,867022,181673,1
20744,867022,1181,1


In [ ]:
edges_df = edges_df.sort_values(['from_title', 'to_title'])

In [ ]:
edges_df.to_csv('./edges_df.csv', index=False)

In [ ]:
s_id_title.to_csv('./s_id_title.csv', index=False)

In [ ]:
!sha256sum ./s_id_title.csv

f2c7e4cfc9895e3dc1eabc323f6fa6164eb515e5e229394bd2c795cf020ee391  ./s_id_title.csv


In [ ]:
!sha256sum ./edges_df.csv

2b475e5fffb6e5154a3ae3dd6dc63f6b0c5261b50c914b316787a5c1e1c295e6  ./edges_df.csv


# 그래프 데이터 로딩

생성된 그래프 데이터를 불러옵니다.
파일의 체크섬을 확인하여 유효한 데이터인지 확인합니다.

- `!sha256sum ./s_id_title.csv`: `f2c7e4cfc9895e3dc1eabc323f6fa6164eb515e5e229394bd2c795cf020ee391`
- `!sha256sum ./edges_df.csv`: `2b475e5fffb6e5154a3ae3dd6dc63f6b0c5261b50c914b316787a5c1e1c295e6  `

- s_id_title은 행이 867024개인 Series입니다.
- edges_df는 행이 18700924개인 DataFrame입니다.

In [ ]:
!sha256sum ./s_id_title.csv
!sha256sum ./edges_df.csv

f2c7e4cfc9895e3dc1eabc323f6fa6164eb515e5e229394bd2c795cf020ee391  ./s_id_title.csv
2b475e5fffb6e5154a3ae3dd6dc63f6b0c5261b50c914b316787a5c1e1c295e6  ./edges_df.csv


In [ ]:
edges_df = pd.read_csv('./edges_df.csv')
s_id_title = pd.read_csv('./s_id_title.csv', keep_default_na=False)['title']
s_title_id = s_id_title.reset_index().set_index('title', verify_integrity=True)['index'].sort_index()

양방향 연결이 모두 존재하는 경우만을 모은 `edges_undirected_df`를 만듭니다. 약 30초 소요됩니다.

In [ ]:
# Create a reversed version of the original edges
reversed_df = edges_df.rename(columns={'from_title': 'to_title', 'to_title': 'from_title'})

# Perform an inner merge to find mutual (bidirectional) edges
bidirectional = edges_df.merge(reversed_df, on=['from_title', 'to_title'])

# Calculate sum of inverses of counts to set it as weights
bidirectional['weight'] = ((1 / bidirectional['count_x']) + (1 / bidirectional['count_y'])) / 2

# Since both directions should be included, concatenate both directions
edges_undirected_df = pd.concat([
    bidirectional[['from_title', 'to_title', 'weight']],
    bidirectional[['to_title', 'from_title', 'weight']].rename(columns={'to_title': 'from_title', 'from_title': 'to_title'})
])

# Drop duplicates just in case, and sort
edges_undirected_df = edges_undirected_df.drop_duplicates().sort_values(['from_title', 'to_title']).reset_index(drop=True)


# `from_title`열에서 이분탐색

`from_title`열에서 이분탐색을 수행합니다. 정렬된 `from_title`열을 기준으로 `target`과 일치하는 부분의 시작과 끝을 이분탐색으로 구합니다.

In [ ]:
def find_lower_bound(target, _edges_df):
    left, right = 0, len(_edges_df)
    while left < right:
        mid = (left + right) // 2
        if _edges_df.iloc[mid]['from_title'] < target:
            left = mid + 1
        else:
            right = mid
    return left

def find_upper_bound(target, _edges_df):
    left, right = 0, len(_edges_df)
    while left < right:
        mid = (left + right) // 2
        if _edges_df.iloc[mid]['from_title'] <= target:
            left = mid + 1
        else:
            right = mid
    return left

def binary_search_title(target, _edges_df):
    """
    Perform binary search on a DataFrame sorted by 'from_title' to find all rows where from_title == target.

    Parameters:
        target (int): The title id to search for.

    Returns:
        pd.DataFrame: Subset of rows with from_title == target. Empty if not found.
    """

    lower = find_lower_bound(target, _edges_df)
    upper = find_upper_bound(target, _edges_df)

    if lower == upper:
        return pd.DataFrame(columns=_edges_df.columns)  # target not found

    return _edges_df.iloc[lower:upper]


# 문서 A -> B 최단 경로를 찾는 BFS 알고리즘

- BFS 알고리즘을 활용하여 나무위키 문서 A에서 문서 B로 링크를 타고 이동하는 최단 경로를 찾습니다.
- 문서 제목을 이분탐색으로 찾습니다.
- 최단 거리 경로가 여러 개이면 모두 구합니다.
- `undirected`가 `True`이면 간선이 두 방향 모두 존재하는 경우만을 고려합니다.

In [ ]:
def bfs_shortest_path(start_title, target_title, undirected):
    """
    Finds the shortest path between two documents using BFS (layer-by-layer).

    Parameters:
        start_title (str): Title of the start document.
        target_title (str): Title of the target document.
        undirected (bool): If True, treat the graph as undirected.

    Returns:
        List[str] or None: List of titles from start to target, or None if no path exists.
    """
    start_id = s_title_id.get(start_title)
    target_id = s_title_id.get(target_title)
    if pd.isna(start_id) or pd.isna(target_id):
        return None

    if start_id == target_id:
        return [start_id]

    visited = set([start_id])
    queue = [[start_id]]
    result = []
    distance = 1

    _edges_df = edges_undirected_df if undirected else edges_df

    while queue:
        next_queue = []
        curr_visited = set()
        distance += 1
        print(f"searching for {distance=}")

        for path in tqdm(queue):
            edges = binary_search_title(path[-1], _edges_df)

            for row in edges.itertuples(index=False):
                neighbor = row.to_title
                if neighbor in visited:
                    continue

                curr_visited.add(neighbor)
                new_path = path + [neighbor]

                if neighbor == target_id:
                    result.append(new_path)
                else:
                    next_queue.append(new_path)
        if result:
            return [list(map(lambda x: s_id_title.loc[x], path)) for path in result]
        # Move to next BFS layer
        queue = next_queue
        visited |= curr_visited

    return None  # No path found

In [ ]:
bfs_shortest_path('수학', '전투기', False)

searching for distance=2


100%|██████████| 1/1 [00:00<00:00, 64.35it/s]


searching for distance=3


100%|██████████| 225/225 [00:00<00:00, 312.45it/s]


searching for distance=4


100%|██████████| 14361/14361 [00:32<00:00, 445.73it/s]


[['수학', '한국십진분류법/요목표', '공군', '전투기'],
 ['수학', '한국십진분류법/요목표', '유럽', '전투기'],
 ['수학', '한국십진분류법/요목표', '종이', '전투기'],
 ['수학', '한국십진분류법/요목표', '프랑스', '전투기'],
 ['수학', 'N(포켓몬스터)', '초음파', '전투기'],
 ['수학', 'Q.E.D. 증명종료', '2009년', '전투기'],
 ['수학', '공과대학', '대한민국 공군', '전투기'],
 ['수학', '공대개그', '관제탑/유머', '전투기'],
 ['수학', '공대개그', '애국가', '전투기'],
 ['수학', '공대개그', '종이', '전투기'],
 ['수학', '공대개그', '티타늄', '전투기'],
 ['수학', '공업수학', 'F', '전투기'],
 ['수학', '공업수학', '보잉', '전투기'],
 ['수학', '과학', '야마하', '전투기'],
 ['수학', '노벨경제학상', '프랑스', '전투기'],
 ['수학', '노벨상', '베트남 전쟁', '전투기'],
 ['수학', '노벨상', '유럽', '전투기'],
 ['수학', '노벨상', '제1차 세계 대전', '전투기'],
 ['수학', '노벨상', '프랑스', '전투기'],
 ['수학', '논리학', '유럽', '전투기'],
 ['수학', '대기과학', '파리(프랑스)', '전투기'],
 ['수학', '동양', '프랑스', '전투기'],
 ['수학', '리처드 파인만', '1965년', '전투기'],
 ['수학', '리처드 파인만', 'NASA', '전투기'],
 ['수학', '리처드 파인만', '파리(프랑스)', '전투기'],
 ['수학', '물리학', '성층권', '전투기'],
 ['수학', '물리학', '우주에서는 총알이 발사되지 않는다', '전투기'],
 ['수학', '물리학', '필립 모티머 교수', '전투기'],
 ['수학', '바이오쇼크 2', '드릴', '전투기'],
 ['수학', '바이오쇼크 2', '

In [ ]:
bfs_shortest_path('수학', '전투기', True)

searching for distance=2


100%|██████████| 1/1 [00:00<00:00, 253.88it/s]


searching for distance=3


100%|██████████| 50/50 [00:00<00:00, 703.49it/s]


searching for distance=4


100%|██████████| 928/928 [00:01<00:00, 696.38it/s]


searching for distance=5


100%|██████████| 22014/22014 [00:35<00:00, 624.98it/s]


[['수학', '경제학', '미국', '미군', '전투기'],
 ['수학', '과학', '미국', '미군', '전투기'],
 ['수학', '노벨상', '미국', '미군', '전투기'],
 ['수학', '노벨상', '영국', '공군', '전투기'],
 ['수학', '노벨상', '한국인', '베트남 전쟁', '전투기'],
 ['수학', '물리학', '광속', '음속', '전투기'],
 ['수학', '버트런드 러셀', '제1차 세계 대전', '영국 본토 항공전', '전투기'],
 ['수학', '버트런드 러셀', '제1차 세계 대전', '폭격기', '전투기'],
 ['수학', '언어', '영국', '공군', '전투기'],
 ['수학', '언어', '캐나다', 'F-35', '전투기'],
 ['수학', '언어', '캐나다', 'F/A-18', '전투기'],
 ['수학', '언어', '한국인', '베트남 전쟁', '전투기'],
 ['수학', '이산수학', '셈 측도', '말(동물)', '전투기']]

# 문서 A -> B 최단 경로를 찾는 다익스트라 알고리즘

In [ ]:
def dijkstra_shortest_path(start_title, target_title, undirected):
    """
    Finds the shortest path between two documents using Dijkstra's algorithm.
    Edge weights are defined as 1 / count.

    Parameters:
        start_title (str): Title of the start document.
        target_title (str): Title of the target document.

    Returns:
        List[str] or None: List of titles from start to target, or None if no path exists.
    """
    start_id = s_title_id.get(start_title)
    target_id = s_title_id.get(target_title)
    if pd.isna(start_id) or pd.isna(target_id):
        return None

    # Priority queue: (cost, current_node, path)
    heap = [(0, start_id, [start_id])]
    visited = set()

    _edges_df = edges_undirected_df if undirected else edges_df
    pbar = tqdm(total=len(s_id_title), desc="Dijkstra Search", unit="node")

    while heap:
        cost, node, path = heapq.heappop(heap)
        if node == target_id:
            pbar.close()
            return cost, [s_id_title.loc[pid] for pid in path]

        if node in visited:
            continue
        visited.add(node)
        pbar.update(1)

        edges = binary_search_title(node, _edges_df)
        for row in edges.itertuples(index=False):
            neighbor = row.to_title
            weight = row.weight if undirected else 1/row.count
            if neighbor not in visited:
                heapq.heappush(heap, (cost + weight, neighbor, path + [neighbor]))

    pbar.close()
    return None  # No path found


In [ ]:
dijkstra_shortest_path('수학', '전투기', False)

Dijkstra Search:   1%|          | 4586/867024 [00:11<36:39, 392.07node/s]


(1.2083333333333333, ['수학', '의학', '의사', '군의관', '대한민국 공군', '전투기'])